In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src

In [ ]:
import os
import cv2
import json
import glob
import torch
import pydicom
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn.functional as F

from collections import Counter
from tqdm import tqdm


pd.set_option('display.width', 5000)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
from params import *
from data.dataset import *
from data.transforms import *
from data.preparation import *
from model_zoo.models import define_model
from util.torch import load_model_weights
from inference.lvl1 import Config, predict

### Inference

In [ ]:
EXP_FOLDER = "../logs/2024-08-29/0/"  # coatnet_rmlp_2_rw_384 aug 50 ep new folds
# EXP_FOLDER = "../output/2024-08-29_0/"  # coatnet_rmlp_2_rw_384 aug 50 ep new folds

FOLD = 0

In [ ]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))

df = prepare_coords_data(config.coords_folder, use_ext=config.use_ext)

folds = pd.read_csv(config.folds_file)
df = df.merge(folds, how="left")
df['fold'] = df['fold'].fillna(-1)

In [ ]:
for fold in range(4):

    model_fct = define_model
    if hasattr(config, "use_decoder"):
        if config.use_decoder:
            model_fct = define_model_dec

    model = model_fct(
        config.name,
        drop_rate=config.drop_rate,
        drop_path_rate=config.drop_path_rate,
        pooling=config.pooling if hasattr(config, "pooling") else "avg",
        head_3d=config.head_3d,
        n_frames=config.n_frames,
        num_classes=config.num_classes,
        num_classes_aux=config.num_classes_aux,
        n_channels=config.n_channels,
        reduce_stride=config.reduce_stride,
        pretrained=False,
    )
    model = model.cuda().eval()

    weights = EXP_FOLDER + f"{config.name}_{fold}.pt"
    try:
        model = load_model_weights(model, weights, verbose=config.local_rank == 0)
    except FileNotFoundError:
        continue

    df_val = df[df['fold'] == fold].reset_index(drop=True)
    dataset = CoordsDataset(df_val, transforms=get_transfos(augment=False, resize=config.resize, use_keypoints=True))

    preds, _ = predict(model, dataset, config.loss_config, batch_size=32, use_fp16=True)

    np.save(EXP_FOLDER + f"pred_inf_{fold}.npy", preds)

### Eval

In [ ]:
PLOT = False

In [ ]:
df_gt = prepare_data()
df_sev = prepare_data_crop(DATA_PATH)

In [ ]:
ds = []
df_ = prepare_data()

for fold in range(4):
    preds = np.load(EXP_FOLDER + f"pred_inf_{fold}.npy")
    df_val = df[df['fold'] == fold].reset_index(drop=True)
    
    dataset = CoordsDataset(df_val, transforms=get_transfos(augment=False, use_keypoints=True))

    for idx in tqdm(range(len(dataset))):
        study = df_val['study_id'][idx]
        series = df_val['series_id'][idx]

        img, y, _ = dataset[idx]
        labels = np.vstack(df_sev[df_sev['series_id'] == series].sort_values('level')['target'].values)

        gt = df_gt[df_gt['series_id'] == series]
        imgs = np.load(f'../input/npy2/{study}_{series}.npy')

        frame = len(imgs) // 2
        img = imgs[frame]
        img = (img - img.min()) / (img.max() - img.min())

        spinenet_coords = None
        p_ = preds[idx].reshape(-1, 2)
        p = preds[idx].reshape(-1, 2)

        # p = fix_predictions(p, y.numpy())
        p_ = p.copy()

        d = np.abs(p - y.numpy()) * 100
        d = d[y.sum(-1) > 0].mean()
        ds.append(d)

        if PLOT:
            if d > 3:
                y = y[y.sum(-1) > 0]
                # if len(y) == 5:
                #     continue

                print(study, series)
                print('SCS / L-NFN / R-NFN / L-SS / R-SS')
                print(labels)

                # cv2.imwrite(f'../output/fix/{study}_{series}.png', (img * 255).astype(np.uint8))

                plt.figure(figsize=(8, 8))
                plt.imshow(img, cmap="gray")
                plt.scatter(y[:, 0] * img.shape[1], y[:, 1] * img.shape[0], marker="x", label="truth")
                plt.scatter(p_[:, 0] * img.shape[1], p_[:, 1] * img.shape[0], marker="x", label="pred")
                if spinenet_coords is not None:
                    plt.scatter(spinenet_coords[:, 0], spinenet_coords[:, 1], marker="x", label="spinenet")
                plt.title(f'Dist = {d:.2f} - study {study} - series {series}')
                plt.axis(False)
                plt.legend()
                plt.show()

                print(p_[:, 0] * img.shape[1], p_[:, 1] * img.shape[0])

            if idx > 50:
                break
    if PLOT:
        break


In [ ]:
print('Images with error > 5%:', (np.array(ds) > 5).sum())
print('Images with error > 4%:', (np.array(ds) > 4).sum())
print('Images with error > 3%:', (np.array(ds) > 3).sum())
print('Images with error > 3%:', (np.array(ds) > 3).sum())

### Crop
- The following cells need to be run with `FIX = True` and `FIX = False`

In [ ]:
DELTA = 0.1

SAVE = True
PLOT = False

FIX = True

if FIX:
    SAVE_FOLDER = f"../input/coords_crops_{DELTA}_f/"
else:
    SAVE_FOLDER = f"../input/coords_crops_{DELTA}_2/"

os.makedirs(SAVE_FOLDER, exist_ok=True)

In [ ]:
def fix_predictions(preds, y, verbose=0):
    for i in range(len(preds)):
        if y[i].min() < 0:
            continue
        elif np.abs(y[i] - preds[i]).sum() > 0.03:
            preds[i] = y[i]
            if verbose:
                print(f'Fix level {LEVELS[i]}')
    return preds

In [ ]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))

df = prepare_coords_data(config.coords_folder, use_ext=config.use_ext)

folds = pd.read_csv(config.folds_file)
df = df.merge(folds, how="left")
df['fold'] = df['fold'].fillna(-1)

In [ ]:
for fold in range(4):
    pred_val = np.load(EXP_FOLDER + f"pred_inf_{fold}.npy")
    df_val = df[df['fold'] == fold].reset_index(drop=True)

    for idx in tqdm(range(len(df_val))):
        study_series = df_val["img_path"][idx].split('/')[-1][:-4]
        imgs_path = DATA_PATH + "npy2/" + study_series + ".npy"      ###### NPY2 ??
        imgs = np.load(imgs_path)
        img = imgs[0]

        y = df_val["target_rel"][idx].copy()

        preds = pred_val[idx].reshape(-1, 2).copy()

        if FIX:
            preds = fix_predictions(preds, y, verbose=0)

        assert preds.min() >= 0, preds
        assert preds.max() <= 1, preds

        crops = np.concatenate([preds, preds], -1)
        crops[:, [0, 1]] -= DELTA
        crops[:, [2, 3]] += DELTA
        crops = crops.clip(0, 1)
        crops[:, [0, 2]] *= imgs.shape[2]
        crops[:, [1, 3]] *= imgs.shape[1]
        crops = crops.astype(int)

        # print(df_val["series_id"][idx])

        if SAVE:
            for i, (x0, y0, x1, y1) in enumerate(crops):
                crop = imgs[:, y0: y1, x0: x1].copy()
                assert crop.shape[2] > 1 and crop.shape[1] > 1
                np.save(SAVE_FOLDER + f'{study_series}_{LEVELS_[i]}.npy', crop)

        if PLOT:
            preds[:, 0] *= imgs.shape[2]
            preds[:, 1] *= imgs.shape[1]

            y[:, 0] *= imgs.shape[2]
            y[:, 1] *= imgs.shape[1]

            plt.figure(figsize=(8, 8))
            plt.imshow(imgs[len(imgs) // 2], cmap="gray")
            plt.scatter(preds[:, 0], preds[:, 1], marker="x", label="center")
            plt.scatter(y[:, 0], y[:, 1], marker="x", label="center gt")
            plt.title(study_series)
            plt.axis(False)
            plt.legend()
            plt.show()


            break

Done